# Ex4
Eliav Dayanof 208674556
Chaya Lasry, 209911379

In [ ]:
import nltk
from collections import Counter, defaultdict, namedtuple
from networkx import DiGraph
from nltk.corpus import dependency_treebank
import graphviz
import os
import random
import numpy as np
from networkx.drawing.nx_agraph import write_dot, read_dot
from networkx import minimum_spanning_arborescence
from nltk.parse import DependencyGraph
from networkx.drawing.nx_agraph import to_agraph
import time
from sklearn.externals._packaging.version import Tuple
import networkx as nx

nltk.download('dependency_treebank')
random.seed(10)

In [ ]:
sentences = dependency_treebank.parsed_sents()
k = len(sentences) // 10
train, test = sentences[:-k], sentences[-k:]
print(len(train))
from Chu_Liu_Edmonds_algorithm import min_spanning_arborescence_nx


def triples_to_counter(triples, lr=1):
    c = Counter()
    for x in triples:
        z = zip(x[0], x[-1])
        for i in range(lr):
            c.update(z)
    return c


def edges_to_counter(edges, dg, lr=1):
    c = Counter()
    for i, j in edges:
        c.update(inds_to_feats(i, j, dg))
    return c


def score(features, weights):
    return sum(weights[f] for f in features)


def inds_to_feats(i, j, dg):
    return [(dg.nodes[i]['word'], dg.nodes[j]['word']), (dg.nodes[i]['tag'], dg.nodes[j]['tag'])]


def get_weight(i, j, dg, w):
    return score(inds_to_feats(i, j, dg), w)


def get_mst(dg: DependencyGraph, w):
    G = DiGraph()
    # nodes = dg.nodes.values()
    import itertools
    for i, j in itertools.product(range(len(dg.nodes)), range(len(dg.nodes))):
        if i != j and j != 0:
            G.add_edge(i, j, weight=-get_weight(i, j, dg, w))
    # for i, n1 in enumerate(nodes):
    #     for j, n2 in enumerate(nodes):
    return minimum_spanning_arborescence(G) if len(dg.nodes) else dg


def perceptron(lr=1, max_iter=2):
    w, total_w = Counter(), Counter()
    st = time.time()
    for r in range(max_iter):
        random.shuffle(train)
        for i, dg in enumerate(train):
            T_prime = get_mst(dg, w)
            print(f"Iteration {i}:\n"
                  f"Current runtime: {time.time() - st}\n")
            w.update(triples_to_counter(dg.triples(), lr))
            w.subtract(edges_to_counter(T_prime.edges(), dg, lr))
            total_w.update(w)
    print(f"runtime: {time.time() - st} seconds\n")
    return total_w


def train_perceptron(lr=1, max_iter=2):
    weights = perceptron(lr, max_iter)
    for key in weights:
        weights[key] /= max_iter * len(train)
    return weights



In [ ]:
mst_counter = train_perceptron()
mst_counter

In [ ]:
def loss(t1_gold: DependencyGraph, t2_pred: DiGraph) -> int:
    gold_nodes = t1_gold.nodes
    l2 = sum(gold_nodes[e[1]]['head'] == e[0] for e in t2_pred.edges) / (len(gold_nodes) - 1)
    return l2


def acc():
    return np.mean([loss(dg, get_mst(dg, mst_counter)) for dg in test])


print(acc()) # 0.3633332601944238